In [ ]:
import pickle
import numpy as np
from collections import defaultdict

# --- USER SETTING ---
PERCENTILE = 48  # flag all windows above this percentile (e.g. 95th percentile)

# --- Load Results ---
with open('50_5_42.pkl', 'rb') as f:
    all_results = pickle.load(f)

# --- Compute Anomaly Scores ---
anomaly_scores = defaultdict(lambda: {'score_sum': 0.0, 'count': 0})

for iteration_result in all_results:
    buckets = iteration_result['buckets']
    bucket_results = iteration_result['bucket_results']

    for bucket_result in bucket_results:
        bucket_idx = bucket_result['bucket_idx']
        final_results = bucket_result['final_results']
        indices_in_bucket = buckets[bucket_idx]

        mean = np.mean(final_results)
        std = np.std(final_results) if np.std(final_results) != 0 else 1e-8

        for i, idx in enumerate(indices_in_bucket):
            sim = final_results[i]
            deviation = abs(sim - mean) / std
            anomaly_scores[idx]['score_sum'] += deviation
            anomaly_scores[idx]['count'] += 1

# --- Final Score Calculation ---
final_scores = {
    idx: score_data['score_sum'] / score_data['count']
    for idx, score_data in anomaly_scores.items()
}

# --- Compute Threshold by Percentile ---
all_score_values = list(final_scores.values())
threshold = np.percentile(all_score_values, PERCENTILE)

# --- Flag Anomalous Windows ---
anomalous_windows = {
    idx: score for idx, score in final_scores.items() if score >= threshold
}

print(f"\n{len(anomalous_windows)} anomalous windows detected above the {PERCENTILE}th percentile (threshold ≈ {threshold:.4f}):\n")
for idx, score in sorted(anomalous_windows.items()):
    print(f"Window {idx} - Score: {score:.4f}")

# --- Save to file ---
with open("flagged_anomalous_windows.txt", "w") as f:
    for idx, score in sorted(anomalous_windows.items()):
        f.write(f"Window {idx} - Score: {score:.4f}\n")

print("\nSaved to 'flagged_anomalous_windows.txt'")



125 anomalous windows detected above the 35th percentile (threshold ≈ 0.7981):

Window 0 - Score: 0.8057
Window 1 - Score: 0.8220
Window 7 - Score: 0.8129
Window 10 - Score: 0.8065
Window 14 - Score: 0.8120
Window 17 - Score: 0.8197
Window 19 - Score: 0.8109
Window 20 - Score: 0.8260
Window 21 - Score: 0.8175
Window 23 - Score: 0.8827
Window 24 - Score: 0.9159
Window 25 - Score: 0.8804
Window 26 - Score: 0.8783
Window 27 - Score: 0.9068
Window 28 - Score: 0.8716
Window 29 - Score: 0.8463
Window 30 - Score: 0.8516
Window 31 - Score: 0.8514
Window 32 - Score: 0.8339
Window 33 - Score: 0.8199
Window 34 - Score: 0.8024
Window 35 - Score: 0.8096
Window 36 - Score: 0.8056
Window 42 - Score: 0.8169
Window 45 - Score: 0.8570
Window 46 - Score: 0.8184
Window 48 - Score: 0.8296
Window 49 - Score: 0.8227
Window 50 - Score: 0.8162
Window 51 - Score: 0.8318
Window 52 - Score: 0.8732
Window 53 - Score: 0.8198
Window 54 - Score: 0.8346
Window 55 - Score: 0.8373
Window 57 - Score: 0.8039
Window 58 - 